In [1]:
import os
import json
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

def diagnosticar_problema_metadatos(csv_path, base_folder):
    """
    Diagnostica problemas de inconsistencia entre imágenes y metadatos.
    
    Args:
        csv_path: Ruta al archivo CSV con los datos originales
        base_folder: Carpeta base del dataset (donde están 'data' y 'sets')
    """
    print("DIAGNÓSTICO DE INCONSISTENCIAS EN METADATOS")
    print("===========================================")
    
    # 1. Cargar el CSV original
    print(f"\n1. Cargando CSV original: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"   Número de filas en el CSV: {len(df)}")
    print(f"   Primeros 5 nombres de archivo en el CSV:")
    for i, file_name in enumerate(df['Archivo_Imagen'].head(5)):
        print(f"     {i+1}. {file_name}")
    
    # 2. Verificar archivos en carpeta 'data'
    data_folder = os.path.join(base_folder, "data")
    print(f"\n2. Verificando archivos en carpeta 'data': {data_folder}")
    if os.path.exists(data_folder):
        data_files = [f for f in os.listdir(data_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"   Número de imágenes en 'data': {len(data_files)}")
        print(f"   Primeros 5 archivos en 'data':")
        for i, file_name in enumerate(sorted(data_files)[:5]):
            print(f"     {i+1}. {file_name}")
    else:
        print(f"   ¡ERROR! La carpeta 'data' no existe: {data_folder}")
        return
    
    # 3. Verificar si los nombres de archivo del CSV existen en 'data'
    print("\n3. Verificando correspondencia entre CSV y carpeta 'data':")
    csv_files = set(df['Archivo_Imagen'])
    data_files_set = set(data_files)
    csv_not_in_data = csv_files - data_files_set
    data_not_in_csv = data_files_set - csv_files
    
    print(f"   Archivos en CSV pero no en 'data': {len(csv_not_in_data)}")
    if csv_not_in_data:
        print(f"   Ejemplos (hasta 5):")
        for i, file_name in enumerate(list(csv_not_in_data)[:5]):
            print(f"     {i+1}. {file_name}")
    
    print(f"   Archivos en 'data' pero no en CSV: {len(data_not_in_csv)}")
    if data_not_in_csv:
        print(f"   Ejemplos (hasta 5):")
        for i, file_name in enumerate(list(data_not_in_csv)[:5]):
            print(f"     {i+1}. {file_name}")
    
    # 4. Examinar los metadatos en conjuntos
    sets_folder = os.path.join(base_folder, "sets")
    set_names = ["train", "val", "test"]
    
    for set_name in set_names:
        set_folder = os.path.join(sets_folder, set_name)
        metadata_path = os.path.join(set_folder, "metadata.jsonl")
        
        print(f"\n4. Examinando conjunto '{set_name}':")
        
        if not os.path.exists(set_folder):
            print(f"   ¡La carpeta '{set_name}' no existe!")
            continue
            
        # Listar imágenes en la carpeta
        set_files = [f for f in os.listdir(set_folder) 
                    if os.path.isfile(os.path.join(set_folder, f)) 
                    and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"   Número de imágenes en '{set_name}': {len(set_files)}")
        
        # Verificar archivo de metadatos
        if not os.path.exists(metadata_path):
            print(f"   ¡El archivo metadata.jsonl no existe en '{set_name}'!")
            continue
            
        # Leer metadatos
        metadata_entries = []
        with open(metadata_path, 'r') as f:
            for line in f:
                metadata_entries.append(json.loads(line))
        
        print(f"   Número de entradas en metadata.jsonl: {len(metadata_entries)}")
        
        # Verificar correspondencia
        metadata_files = set(entry["file_name"] for entry in metadata_entries)
        set_files_set = set(set_files)
        
        metadata_not_in_folder = metadata_files - set_files_set
        folder_not_in_metadata = set_files_set - metadata_files
        
        print(f"   Archivos en metadata pero no en carpeta: {len(metadata_not_in_folder)}")
        print(f"   Archivos en carpeta pero no en metadata: {len(folder_not_in_metadata)}")
        
        # 5. Verificar ejemplo concreto - muestra una imagen y sus metadatos
        if set_files:
            sample_file = set_files[0]  # Tomar la primera imagen
            print(f"\n5. Análisis detallado para un ejemplo en '{set_name}': {sample_file}")
            
            # Buscar en CSV
            csv_row = df[df['Archivo_Imagen'] == sample_file]
            if not csv_row.empty:
                print(f"   Datos del CSV para esta imagen:")
                for column in ['Nombre', 'Apellido Paterno', 'Apellido Materno', 'CURP']:
                    print(f"     {column}: {csv_row.iloc[0][column]}")
            else:
                print(f"   ¡Esta imagen no está en el CSV!")
            
            # Buscar en metadata
            sample_metadata = None
            for entry in metadata_entries:
                if entry["file_name"] == sample_file:
                    sample_metadata = entry
                    break
                    
            if sample_metadata:
                print(f"   Datos del metadata.jsonl para esta imagen:")
                metadata_data = json.loads(sample_metadata["ground_truth"])["gt_parse"]
                for key in ['nombre', 'apellido_paterno', 'apellido_materno', 'curp']:
                    print(f"     {key}: {metadata_data.get(key, 'N/A')}")
            else:
                print(f"   ¡Esta imagen no está en metadata.jsonl!")
                
            # Comparar
            if csv_row.empty or not sample_metadata:
                print("   No se puede comparar: faltan datos")
            else:
                print("\n   Comparación CSV vs metadata:")
                csv_data = csv_row.iloc[0]
                metadata_data = json.loads(sample_metadata["ground_truth"])["gt_parse"]
                
                matches = {
                    'Nombre': csv_data['Nombre'] == metadata_data.get('nombre', ''),
                    'Apellido Paterno': csv_data['Apellido Paterno'] == metadata_data.get('apellido_paterno', ''),
                    'Apellido Materno': csv_data['Apellido Materno'] == metadata_data.get('apellido_materno', ''),
                    'CURP': csv_data['CURP'] == metadata_data.get('curp', '')
                }
                
                for field, match in matches.items():
                    status = "✓ COINCIDE" if match else "✗ NO COINCIDE"
                    print(f"     {field}: {status}")
    
    print("\n===========================================")
    print("DIAGNÓSTICO COMPLETADO")

# Ejemplo de uso
if __name__ == "__main__":
    csv_path = "datos_ines.csv"  # Ruta al CSV original
    base_folder = r"C:\Users\Adrian\Desktop\INEs Dataset"  # Carpeta base
    
    diagnosticar_problema_metadatos(csv_path, base_folder)

DIAGNÓSTICO DE INCONSISTENCIAS EN METADATOS

1. Cargando CSV original: datos_ines.csv
   Número de filas en el CSV: 300
   Primeros 5 nombres de archivo en el CSV:
     1. ine_falsa_001.jpg
     2. ine_falsa_002.jpg
     3. ine_falsa_003.jpg
     4. ine_falsa_004.jpg
     5. ine_falsa_005.jpg

2. Verificando archivos en carpeta 'data': C:\Users\Adrian\Desktop\INEs Dataset\data
   Número de imágenes en 'data': 300
   Primeros 5 archivos en 'data':
     1. ine_falsa_001.jpg
     2. ine_falsa_002.jpg
     3. ine_falsa_003.jpg
     4. ine_falsa_004.jpg
     5. ine_falsa_005.jpg

3. Verificando correspondencia entre CSV y carpeta 'data':
   Archivos en CSV pero no en 'data': 0
   Archivos en 'data' pero no en CSV: 0

4. Examinando conjunto 'train':
   Número de imágenes en 'train': 210
   Número de entradas en metadata.jsonl: 210
   Archivos en metadata pero no en carpeta: 0
   Archivos en carpeta pero no en metadata: 0

5. Análisis detallado para un ejemplo en 'train': ine_falsa_001.jpg
  